<a href="https://colab.research.google.com/github/Prax0028/Violence_Detection_Model_Codes/blob/main/CNN_Hyper_ParameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Prax0028/violence_dataset

Cloning into 'violence_dataset'...
remote: Enumerating objects: 10971, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 10971 (delta 7), reused 0 (delta 0), pack-reused 10956 (from 1)
Receiving objects: 100% (10971/10971), 645.81 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (11065/11065), done.


In [ ]:
!pip install -U albumentations==1.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [ ]:
!pip install --upgrade opencv-contrib-python

In [ ]:
import sys
sys.path.append('/content/violence_dataset')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms as T

from tqdm import tqdm
import albumentations as A
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import utils

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
from itertools import product
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A

# Device setup
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()

    self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling to reduce spatial dimensions to 1x1
        )


    self.classifier = nn.Sequential(
            nn.Flatten(),  # Flatten the feature maps into a 1D vector
            nn.Linear(128, 512),  # Assuming input size is (128, 128, 3), output after conv layers will be (128, 16, 16)
            nn.ReLU(),
            nn.Linear(512, 2)  # Output layer with 2 units (for 'violence' and 'non-violence')
    )

    self.gradient = None


  def forward(self,images):

    x = self.feature_extractor(images) #activation_maps
    x = self.classifier(x)

    return x


In [ ]:

# Training Function
def train_fun(dataloader, model, optimizer, criterion):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in tqdm(dataloader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy

# Evaluation Function
def eval_fun(dataloader, model, criterion):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_actual_labels = []
    all_predicted_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            logits = model(images)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)
            all_actual_labels.extend(labels.cpu().numpy())
            all_predicted_labels.extend(predictions.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy, all_actual_labels, all_predicted_labels


In [ ]:

# Data Augmentation
train_augs = A.Compose([
    A.Resize(224, 224),
    A.Rotate(),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_augs = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:

# Hyperparameter Grid
# Batch sizes: Small batch sizes can provide better generalization but may be noisier.
batch_sizes = [2, 4, 8, 16, 32]

# Learning rates: Typical ranges for CNNs. Experimenting with different magnitudes helps stabilize training.
learning_rates = [0.001, 0.0005, 0.0001]

# Train-validation splits: Commonly used splits to test model generalization.
train_val_splits = [0.7, 0.75, 0.8, 0.85, 0.9]

# Optimizers: Different optimizers can affect convergence speed and final performance.
#optimizers = ['Adam', 'SGD', 'RMSprop', 'AdamW']
optimizers = ['Adam']

# Results List
results = []


In [ ]:
import os

In [ ]:

# Load Data
data = pd.read_csv('/content/violence_dataset/image_labels.csv')  # Update this path as needed

# Iterate over all combinations of hyperparameters
for batch_size, lr, split, optimizer_name in product(batch_sizes, learning_rates, train_val_splits, optimizers):
    # Data Split
    train_df, valid_df = train_test_split(data, test_size=1-split, random_state=42)
    trainset = utils.ImageDataset(train_df, augs=train_augs, data_dir='/content/violence_dataset/')
    validset = utils.ImageDataset(valid_df, augs=valid_augs, data_dir='/content/violence_dataset/')

    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    validloader = DataLoader(validset, batch_size=batch_size)

    # Initialize model
    model = CNNModel().to(DEVICE)

    # Set optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr) if optimizer_name == 'Adam' else optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # Train for 5 epochs
    for epoch in range(5):
        train_loss, train_accuracy = train_fun(trainloader, model, optimizer, criterion)
        valid_loss, valid_accuracy, _, _ = eval_fun(validloader, model, criterion)

        # Only log final accuracy after 20 epochs
        if epoch == 4:
            results.append({
                'batch_size': batch_size,
                'learning_rate': lr,
                'train_val_split': split,
                'optimizer': optimizer_name,
                'train_accuracy': train_accuracy,
                'valid_accuracy': valid_accuracy
            })
            # Print final accuracies
            print(f"Final Training Accuracy: {train_accuracy:.4f}, Final Validation Accuracy: {valid_accuracy:.4f}")

        # Save to CSV after each hyperparameter combination finishes
    results_df = pd.DataFrame(results)
    results_df.to_csv('hyperparameter_results.csv', mode='a', index=False, header=not os.path.isfile('hyperparameter_results.csv'))
    print(f"Saved results to CSV for batch_size={batch_size}, lr={lr}, split={split}, optimizer={optimizer_name}")


 62%|██████▏   | 2403/3872 [07:42<04:42,  5.19it/s]


KeyboardInterrupt: 